In [1]:
# Necessary Imports
import pandas as pd
import requests

In [2]:
# File handler class
# Reads and writes CSV Files
class FileHandler:
    fileName = ""
    
    readHandle = pd.DataFrame({})
    
    def __init__(self, fileName):
        self.fileName = fileName
    
    def readFile(self):
        self.readHandle = pd.read_csv(self.fileName)
        
    
    def getReadHandle(self):
        return self.readHandle
    
    def writeFile(self, df):
        df.to_csv(self.fileName, encoding='utf-8', index=False)

In [3]:
# Similar to a POJO class in Java
# Stores the structure one cryptocurrency row
class CryptoRow:
    unixTimeStamp = "" #Timestamp
    date = "" #Date format of the timestamp
    symbol = "" # Example : ETHUSD
    openValue = 0.0 # Value at which the cryptocurrency opened
    highValue = 0.0 # Highest Value that the cryptocurrency reached
    lowValue = 0.0 # Lowest Value that the cryptocurrency reached
    closeValue = 0.0 # Value at which the cryptocurrency closed for a particular day
    
    columns = ["Unix Timestamp", "Date", "Symbol", "Open", "High", "Low", "Close"]
        
    def _init_(self, unixTimeStamp, date, symbol, openValue, highValue, lowValue, closeValue):
        self.setUnixTimeStamp(unixTimeStamp)
        self.setDate(date)
        self.setSymbol(symbol)
        self.setOpenValue(openValue)
        self.setHighValue(highValue)
        self.setLowValue(lowValue)
        self.setCloseValue(closeValue)
        
    def getDataFrameObject(self, obj):
        columns = CryptoRow.columns
        jsonObj = {}
        index = []
        counter = 1;
        for column in columns:
            jsonObj[column] = []
        for row in obj:
            jsonObj["Unix Timestamp"].append(str(row.getUnixTimeStamp()))
            jsonObj["Date"].append(row.getDate())
            jsonObj["Symbol"].append(row.getSymbol())
            jsonObj["Open"].append(row.getOpenValue())
            jsonObj["High"].append(row.getHighValue())
            jsonObj["Low"].append(row.getLowValue())
            jsonObj["Close"].append(row.getCloseValue())
            index.append(counter)
            counter = counter + 1

        dfc = DataFrameConstructor(index, jsonObj)
        dfc.construct()
        df = dfc.getDataFrame()
        df = stripDateFrame(df)
        return df

    def stripDateFrame(self, df):
        df['Date'] = pd.to_datetime(df['Date'], errors = 'coerce')
        df['Day'] = pd.DatetimeIndex(df['Date']).day
        df['Month'] = pd.DatetimeIndex(df['Date']).month
        df['Year'] = pd.DatetimeIndex(df['Date']).year
        df['Hour'] = pd.DatetimeIndex(df['Date']).hour
        df['Minute'] = pd.DatetimeIndex(df['Date']).minute
        df.drop(columns=["Date"], inplace=True)
        return df
    
    def setUnixTimeStamp(self, unixTimeStamp):
        self.unixTimeStamp = unixTimeStamp
    def getUnixTimeStamp(self):
        return self.unixTimeStamp
    
    def setDate(self, date):
        self.date = date
    def getDate(self):
        return self.date
    
    def setSymbol(self, symbol):
        self.symbol = symbol
    def getSymbol(self):
        return self.symbol
    
    def setOpenValue(self, openValue):
        self.openValue = openValue
    def getOpenValue(self):
        return self.openValue
    
    def setHighValue(self, highValue):
        self.highValue = highValue
    def getHighValue(self):
        return self.highValue
    
    def setLowValue(self, lowValue):
        self.lowValue = lowValue
    def getLowValue(self):
        return self.lowValue
    
    def setCloseValue(self, closeValue):
        self.closeValue = closeValue
    def getCloseValue(self):
        return self.closeValue

In [4]:
# API invocation class
# Used to create http connection to the necessary third party end point (Cryptocompare) and get resp
class APIConnector:
    baseUrl = "https://min-api.cryptocompare.com"
    
    # API Urls
    singleSymbolPriceUrl = "/data/price?fsym={0}&tsyms={1}"
    dayPriceUrl = "/data/histoday?fsym={0}&tsym={1}&toTs={2}&limit=1"
    
    #Header Variables
    apiKey = "4e08e70a843a3c45d53536674631208a112ad6eab111ad70a1780062aa3a323b"
    
    def constructUrl(self, url, parameters):
        
        tempUrl = url
        count = 0
        for param in parameters:
            tempUrl = tempUrl.replace("{" + str(count) + "}", param)
            count = count + 1
            
        return tempUrl
        
    def invokeAPI(self, url, parameters):
        
        finalUrl = self.baseUrl + self.constructUrl(url, parameters)
        resp = requests.get(finalUrl)
        return resp

In [5]:
# Class to consturct dataframe
# Input - Data values and Columns
# Ouput - Dataframe
class DataFrameConstructor:
    index = []
    obj = {}
    df = {}
    
    def __init__(self, index, obj):
        self.index = index
        self.obj = obj
    
    def construct(self):
        self.df = pd.DataFrame(self.obj, index = self.index)
    
    def getDataFrame(self):
        return self.df
        